# 뉴스 헤드라인 분류하기 (SageMaker 버전)

> 이 노트북은 SageMaker Notebook Instance 및 conda_pytorch_p310 및 SageMaker Studio의 `Python 3 (PyTorch 1.13 Python 3.9 CPU Optimized)` 커널에서 잘 작동합니다.

이 예제에서는 커스텀 스크립트와 [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) 프레임워크를 사용하여 뉴스 헤드라인 분류기 모델을 훈련할 것입니다.

이 "SageMaker" 노트북은 Amazon SageMaker 훈련 작업에서 모델을 훈련하고 관리되는 실시간 추론 엔드포인트에 배포하는 방법을 보여줄 것입니다.

> ⚠️ 우리는 이미 동반 ["Headline Classifier Local" 노트북](Headline%20Classifier%20Local.ipynb)을 실행했다고 가정합니다. 이 노트북은 노트북 자체에서 훈련과 추론을 실행하는 방법을 보여줍니다.

## 설치 및 설정

로컬 노트북과 마찬가지로 시작하기 전에 위젯 라이브러리가 설정되어 있는지 확인할 것입니다.

🟢 하지만 **로컬 노트북과 달리**, **HF Transformers를 설치할 필요가 없다는 점**에 유의하십시오. 실제 훈련과 추론은 컨테이너화된 작업에서 발생하며 이 커널에서는 발생하지 않기 때문입니다.


In [1]:
%pip install  "sagemaker>2.140,<3"

  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.17-py310-none-any.whl.metadata (7.2 kB)
Using cached dill-0.3.9-py3-none-any.whl (119 kB)
Using cached multiprocess-0.70.17-py310-none-any.whl (134 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.2.0 requires dill<0.3.9,>=0.3.0, but you have dill 0.3.9 which is incompatible.
datasets 3.2.0 requires multiprocess<0.70.17, but you have multiprocess 0.70.17 which is incompatible.
Note: you may need to restart the kernel to use updated pack

In [2]:
%%sh
pip install --upgrade jupyter jupyterlab
pip install --upgrade ipywidgets

  Using cached jupyterlab-4.3.3-py3-none-any.whl.metadata (16 kB)


설치가 완료되면 나머지 노트북에서 사용할 라이브러리와 Python 내장 기능을 로드할 것입니다.

[%autoreload magic](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html)은 로컬 .py 파일로 작업할 때 유용합니다. 각 셀 실행 시 라이브러리를 다시 로드하면 노트북 커널을 다시 시작하지 않고도 로컬로 편집/업데이트된 스크립트를 사용할 수 있기 때문입니다.

🟢 이번에는 로컬 노트북에서 필요하지 않았던 일부 **AWS 라이브러리**를 사용할 것입니다:

- `boto3`, [Python용 범용 AWS SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html)
- `sagemaker`, [Amazon SageMaker용 고수준 Python SDK](https://sagemaker.readthedocs.io/en/stable/)

이 두 라이브러리는 모두 오픈 소스이며 PyPI와 GitHub에 게시되어 있습니다.

In [3]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import os  # Operating system utils e.g. file paths

# External Dependencies:
import boto3  # General AWS SDK for Python
import ipywidgets as widgets  # Interactive prediction widget
import pandas as pd  # Utilities for working with data tables (dataframes)
import sagemaker  # High-level Python SDK for Amazon SageMaker

local_dir = "data"

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/17/24 03:16:25] INFO     Found credentials from IAM Role:                                   ]8;id=393380;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=555697;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## 데이터셋 준비 및 업로드하기

이 예제는 [Registry of Open Data on AWS](https://registry.opendata.aws/fast-ai-nlp/) 공개 리포지토리에서 **FastAi AG News** 데이터셋을 다운로드합니다. 이 데이터셋에는 뉴스 헤드라인과 해당 주제 클래스가 포함된 테이블이 들어 있습니다.

In [4]:
%%time
# Download the AG News data from the Registry of Open Data on AWS.
!mkdir -p {local_dir}
!aws s3 cp s3://fast-ai-nlp/ag_news_csv.tgz {local_dir} --no-sign-request

# Un-tar the AG News data.
!tar zxf {local_dir}/ag_news_csv.tgz -C {local_dir}/ --strip-components=1 --no-same-owner

# Push data partitions to separate subfolders, which is useful for local script debugging later
os.renames(f"{local_dir}/test.csv", f"{local_dir}/test/test.csv")
os.renames(f"{local_dir}/train.csv", f"{local_dir}/train/train.csv")
print("Done!")

download: s3://fast-ai-nlp/ag_news_csv.tgz to data/ag_news_csv.tgz
Done!
CPU times: user 11 ms, sys: 39.6 ms, total: 50.6 ms
Wall time: 1.4 s


데이터를 다운로드하고 추출한 후 아래와 같이 일부 예제를 탐색할 수 있습니다:

In [5]:
column_names = ["CATEGORY", "TITLE", "CONTENT"]
# we use the train.csv only
df = pd.read_csv(f"{local_dir}/train/train.csv", names=column_names, header=None, delimiter=",")
# shuffle the DataFrame rows
df = df.sample(frac=1, random_state=1337)

# Make the (1-indexed) category classes more readable:
class_names = ["Other", "World", "Sports", "Business", "Sci/Tech"]
idx2label = {ix: name for ix, name in enumerate(class_names)}
label2idx = {name: ix for ix, name in enumerate(class_names)}

df = df.replace({"CATEGORY": idx2label})
df.head()

,CATEGORY,TITLE,CONTENT
86110,Sci/Tech,Oracle to drop PeopleSoft suit if tender fails,Oracle Corp. notified Delaware's Court of Chan...
74390,Sci/Tech,"NTT DoCoMo, IBM, Intel team to secure mobile d...",With an eye towards making mobile devices and ...
77491,Sci/Tech,Election Is Crunch Time for U.S. Secret Service,With just days to go before the U.S. president...
27497,Sports,Former Celtic striker Larsson on Barcelona bench,Henrik Larsson was left on the bench by Barcel...
47492,World,Four Suicides Linked to Child Porn Probe (AP),AP - The government will press on with a child...


이 연습에서 우리는 **다음만 사용할 것입니다**:

- 뉴스 기사의 **제목**(헤드라인)을 입력으로 사용합니다.
- **카테고리**를 예측할 대상 변수로 사용합니다.

이 데이터셋에는 아래와 같이 4개의 고르게 분포된 주제 클래스가 포함되어 있습니다.

> ℹ️ **'기타'에 대해:** 원시 데이터셋은 1-4의 숫자로 카테고리를 나타내고, 모델은 0부터 시작하는 숫자를 기대하므로, 데이터 준비를 단순하게 유지하고 클래스의 추가적이고 혼란스러운 숫자 표현을 도입하지 않기 위해 사용되지 않은 '기타' 클래스를 삽입했습니다.

In [6]:
df["CATEGORY"].value_counts()

Sci/Tech    30000
Sports      30000
World       30000
Business    30000
Name: CATEGORY, dtype: int64

지금까지는 새로운 내용이 없습니다...

🟢 SageMaker에서 훈련할 때의 주요 차이점은 **데이터셋을 업로드**해야 한다는 것입니다. [훈련 작업이 액세스할 수 있는 곳](https://docs.aws.amazon.com/sagemaker/latest/dg/model-access-training-data.html)에 업로드해야 합니다.

여기서는 [SageMaker 기본 버킷](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-ex-bucket.html)을 사용하여 데이터를 [Amazon S3](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Welcome.html)에 업로드할 것입니다. 원하는 경우 버킷과 폴더 접두사를 사용자 지정할 수 있습니다. 훈련 데이터와 테스트 데이터를 동일한 폴더에 있는 두 파일이 아닌 별도의 S3 폴더로 분리하는 것이 도움이 될 것입니다.

In [7]:
bucket_name = sagemaker.Session().default_bucket()
s3_prefix = "sm101/news"

s3 = boto3.resource("s3")

s3.Bucket(bucket_name).upload_file(f"{local_dir}/train/train.csv", f"{s3_prefix}/train/train.csv")
train_s3_uri = f"s3://{bucket_name}/{s3_prefix}/train"
print(f"train_s3_uri: {train_s3_uri}")

s3.Bucket(bucket_name).upload_file(f"{local_dir}/test/test.csv", f"{s3_prefix}/test/test.csv")
test_s3_uri = f"s3://{bucket_name}/{s3_prefix}/test"
print(f"test_s3_uri: {test_s3_uri}")

[12/17/24 03:16:28] INFO     Found credentials from IAM Role:                                   ]8;id=900787;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=266354;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=304328;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=657096;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

train_s3_uri: s3://sagemaker-us-east-1-057716757052/sm101/news/train
test_s3_uri: s3://sagemaker-us-east-1-057716757052/sm101/news/test


## 훈련 매개변수 정의하기

우리는 [Hugging Face Hub](https://huggingface.co/models)에서 (비교적 작은) 사전 훈련된 모델을 미세 조정할 것이며, 낮은 수준의 훈련 루프를 처음부터 작성하는 대신 높은 수준의 [Trainer API](https://huggingface.co/docs/transformers/main_classes/trainer)를 사용할 것입니다.

🟢 우리의 훈련 스크립트는 궁극적으로 이전과 유사한 매개변수를 사용할 것이지만, 이번에는 이러한 매개변수를 **훈련 작업 API를 통해 전달**할 것입니다.

여기서는 노트북에서 **JSON 직렬화 가능한 매개변수**를 정의한 다음 이를 사용하여 나중에 `transformers.TrainingArguments`를 빌드할 것입니다:

In [8]:
hyperparameters = {
    "model_id": "amazon/bort",  # ID of the pre-trained model to start from
    "class_names": ",".join(class_names),  # Comma-separated list of category names
    "num_train_epochs": 3,  # This time, we'll actually train for a full 3 epochs
    "per_device_train_batch_size": 32,  # Note this is higher than we could set on local hardware
    "per_device_eval_batch_size": 64,  # Note this is higher than we could set on local hardware
    "warmup_steps": 500,  # Higher than we could set with the reduced local training
}
hyperparameters

{'model_id': 'amazon/bort',
 'class_names': 'Other,World,Sports,Business,Sci/Tech',
 'num_train_epochs': 3,
 'per_device_train_batch_size': 32,
 'per_device_eval_batch_size': 64,
 'warmup_steps': 500}

## 지표 정의하기

우리는 훈련된 모델의 품질을 측정하는 방법을 정의하고, 이 정보를 SageMaker에 표시하여 지표 로깅, 자동 모델 튜닝 및 순위표와 같은 기능을 활성화하고자 합니다.

🟢 우리는 훈련 코드가 평소와 같이 지표를 출력하도록 하고, [정규 표현식을 사용](https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html#define-train-metrics)하여 SageMaker가 작업 로그에서 구조화된 지표를 스크랩하는 방법을 정의할 것입니다:

In [9]:
metric_definitions = [
    {"Name": "Epoch", "Regex": r"'epoch': ([0-9\.\-e]+)"},
    {"Name": "Train:Loss", "Regex": r"'loss': ([0-9\.\-e]+)"},
    {"Name": "Train:LearningRate", "Regex": r"'learning_rate': ([0-9\.\-e]+)"},
    {"Name": "Validation:Loss", "Regex": r"'eval_loss': ([0-9\.\-e]+)"},
    {"Name": "Validation:Accuracy", "Regex": r"'eval_accuracy': ([0-9\.\-e]+)"},
    {"Name": "Validation:F1", "Regex": r"'eval_f1': ([0-9\.\-e]+)"},
    {"Name": "Validation:Precision", "Regex": r"'eval_precision': ([0-9\.\-e]+)"},
    {"Name": "Validation:Recall", "Regex": r"'eval_recall': ([0-9\.\-e]+)"},
    {"Name": "Validation:Runtime", "Regex": r"'eval_runtime': ([0-9\.\-e]+)"},
    {"Name": "Validation:SamplesPerSecond", "Regex": r"'eval_samples_per_second': ([0-9\.\-e]+)"},
    {"Name": "Validation:StepsPerSecond", "Regex": r"'eval_steps_per_second': ([0-9\.\-e]+)"},
]
metric_definitions

[{'Name': 'Epoch', 'Regex': "'epoch': ([0-9\\.\\-e]+)"},
 {'Name': 'Train:Loss', 'Regex': "'loss': ([0-9\\.\\-e]+)"},
 {'Name': 'Train:LearningRate', 'Regex': "'learning_rate': ([0-9\\.\\-e]+)"},
 {'Name': 'Validation:Loss', 'Regex': "'eval_loss': ([0-9\\.\\-e]+)"},
 {'Name': 'Validation:Accuracy', 'Regex': "'eval_accuracy': ([0-9\\.\\-e]+)"},
 {'Name': 'Validation:F1', 'Regex': "'eval_f1': ([0-9\\.\\-e]+)"},
 {'Name': 'Validation:Precision',
  'Regex': "'eval_precision': ([0-9\\.\\-e]+)"},
 {'Name': 'Validation:Recall', 'Regex': "'eval_recall': ([0-9\\.\\-e]+)"},
 {'Name': 'Validation:Runtime', 'Regex': "'eval_runtime': ([0-9\\.\\-e]+)"},
 {'Name': 'Validation:SamplesPerSecond',
  'Regex': "'eval_samples_per_second': ([0-9\\.\\-e]+)"},
 {'Name': 'Validation:StepsPerSecond',
  'Regex': "'eval_steps_per_second': ([0-9\\.\\-e]+)"}]

## SageMaker에서 모델 훈련 및 검증하기

이번에는 노트북 자체와 별도의 인스턴스에서 훈련 프로세스를 실행하기 위해 [SageMaker 훈련 작업](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-training.html)을 생성할 것입니다. 이렇게 하면 장기간 유지되는 노트북 환경과 독립적으로 임시 훈련 인프라의 크기를 적절하게 조정할 수 있습니다.

🟢 실제 훈련 코드는 노트북에서 **[scripts/train.py](scripts/train.py)**로 분리되었으며, 이 스크립트를 통해 [SageMaker Python SDK를 통한 사전 빌드된 Hugging Face Framework 컨테이너](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/index.html)를 사용하여 모델을 훈련하고 배포할 것입니다.

### Amazon SageMaker가 사전 빌드된 컨테이너로 스크립트를 실행하는 방법

AWS는 주요 ML 프레임워크에서 프로젝트 구축을 가속화하는 데 도움이 되는 사전 패키지화된 Docker 이미지 세트를 제공합니다: [SageMaker Framework Containers](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-prebuilt.html).

이러한 컨테이너는 GPU 드라이버, 서빙 스택 구현, 핵심 라이브러리 등과 같은 기본 설정을 처리하므로 우리는 훈련 프로세스와 추론 동작 재정의를 위한 일부 Python 스크립트를 단순히 주입할 수 있습니다. 시작 시 동적으로 설치할 추가 종속성을 지정하기 위해 *requirements.txt* 파일을 제공할 수도 있습니다. 이러한 종속성을 컨테이너 이미지에 빌드할 필요가 없습니다.

**결과적으로 우리의 첫 번째 작업은 스크립트(들)와 런타임 간의 인터페이스를 이해하는 것입니다**: 스크립트는 입력 데이터를 어떻게 읽을까요? 매개변수는 어떻게 읽을까요? 결과는 어디에 저장해야 할까요?

#### 훈련 중 컨테이너

훈련 작업 컨테이너가 시작될 때, **코드와 입력 데이터**는 `/opt/ml` 디렉토리 아래의 **로컬 파일**로 다운로드됩니다. 또한 **훈련된 모델**과 기타 파일 출력을 로컬 파일 시스템에 저장할 것입니다 - 아래와 같이:

```
    /opt/ml
    |-- code
    |   `-- <우리의 스크립트(들)>
    |-- input
    |   |-- config
    |   |   |-- hyperparameters.json
    |   |   `-- resourceConfig.json
    |   `-- data
    |       `-- <채널 이름>
    |           `-- <입력 데이터>
    |-- model
    |   `-- <모델 파일>
    `-- output
        `-- failure
```

##### 입력

* `/opt/ml/input/config`는 프로그램 실행 방식을 제어하는 정보를 포함합니다. `hyperparameters.json`은 하이퍼파라미터 이름과 값의 JSON 형식 사전입니다. 이러한 값은 항상 문자열이므로 변환이 필요할 수 있습니다. `resourceConfig.json`은 분산 훈련에 사용되는 네트워크 레이아웃을 설명하는 JSON 형식 파일입니다. scikit-learn은 분산 훈련을 지원하지 않으므로 여기서는 무시할 것입니다.
* `/opt/ml/input/data/<채널 이름>/` (파일 모드의 경우)는 해당 채널의 입력 데이터를 포함합니다. 채널은 CreateTrainingJob에 대한 호출을 기반으로 생성되지만 일반적으로 채널이 알고리즘이 기대하는 것과 일치하는 것이 중요합니다. 각 채널의 파일은 S3에서 이 디렉토리로 복사되며, S3 키 구조가 표시하는 트리 구조를 유지합니다.
* `/opt/ml/input/data/<채널 이름>_<에포크 번호>` (파이프 모드의 경우)는 주어진 에포크에 대한 파이프입니다. 에포크는 0에서 시작하여 읽을 때마다 1씩 증가합니다. 실행할 수 있는 에포크 수에는 제한이 없지만 다음 에포크를 읽기 전에 각 파이프를 닫아야 합니다.

##### 출력

* `/opt/ml/model/`은 알고리즘이 생성하는 모델을 작성하는 디렉토리입니다. 모델은 원하는 형식으로 작성할 수 있습니다. 단일 파일이거나 전체 디렉토리 트리일 수 있습니다. SageMaker는 이 디렉토리의 모든 파일을 압축된 tar 아카이브 파일로 패키징합니다. 이 파일은 `DescribeTrainingJob` 결과에서 반환된 S3 위치에서 사용할 수 있습니다.
* `/opt/ml/output`은 알고리즘이 작업 실패 이유를 설명하는 `failure` 파일을 작성할 수 있는 디렉토리입니다. 이 파일의 내용은 `DescribeTrainingJob` 결과의 `FailureReason` 필드에 반환됩니다. 성공한 작업의 경우 이 파일을 작성할 이유가 없으므로 무시됩니다.

#### 추가 정보

더 많은 정보를 원하시면 다음을 참조하십시오:

- Hugging Face를 위한 [SageMaker Python SDK 가이드](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/index.html) 및 [API 문서](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html) (PyTorch에 대한 동등한 페이지도 유용할 수 있음).
- GitHub의 [AWS Deep Learning Containers 리포지토리](https://github.com/aws/deep-learning-containers)는 기본 컨테이너 이미지를 정의합니다.
- 훈련 및 서빙을 위한 프레임워크 코드에 대한 자세한 내용은 오픈 소스 [SageMaker Training Toolkit](https://github.com/aws/sagemaker-training-toolkit) 및 [SageMaker Inference Toolkit](https://github.com/aws/sagemaker-inference-toolkit)을 참조하십시오. (일부 프레임워크는 이러한 툴킷의 변형을 사용합니다, 예: [sagemaker-pytorch-training-toolkit](https://github.com/aws/sagemaker-pytorch-training-toolkit))

### (선택사항) 스크립트 테스트

> ℹ️ **참고:** 이 단계는 선택사항입니다. 이 예제에서는 훈련 스크립트가 이미 구축되고 테스트되었기 때문입니다!

작업 스크립트 [train.py](scripts/train.py)가 기본적으로 노트북에서 이전에 수행된 것과 동일한 로직과 프로세스를 따르지만, SageMaker를 위해 수정한 내용을 **테스트**하는 것이 좋을 것입니다.

스크립트의 초기 기능 테스트와 디버깅을 위해 매번 전체 SageMaker 훈련 작업을 시작하고 싶지 않을 수 있습니다: 각각의 새로운 작업이 온디맨드 컴퓨팅 리소스를 시작하는 동안 짧은 지연이 발생하기 때문입니다.

이 프로세스를 가속화할 수 있는 여러 가지 방법이 있습니다. 일반적으로 [SageMaker Warm Pools](https://docs.aws.amazon.com/sagemaker/latest/dg/train-warm-pools.html)나 [SageMaker Local Mode](https://sagemaker.readthedocs.io/en/stable/overview.html?#local-mode)를 추천하지만, 이는 표준 워크샵 환경에서는 사용할 수 없습니다. 대신, CLI를 통해 훈련 스크립트를 호출하여 **노트북 내에서 훈련 작업을 시뮬레이션**할 수 있습니다.

아래 셀의 주석을 해제(`Ctrl`+`/`)하고 실행해볼 수 있습니다 - ⚠️ 하지만 주의하세요: 매우 메모리 집약적이므로, 이전 [Headline Classifier Local 노트북](Headline%20Classifier%20Local.ipynb)의 커널을 먼저 종료하거나 재시작해야 합니다.





In [10]:
# class_names_str = ",".join(class_names)  # Comma-separated list for CLI
# !python3 scripts/train.py \
#     --train data/train \
#     --test data/test \
#     --output_data_dir data/local-output \
#     --model_dir data/local-model \
#     --model_id=amazon/bort --class_names={class_names_str} --train_max_steps=20 \
#     --train_batch_size=8 --eval_batch_size=16 --fp16=0

### 작업 생성하기

실제 [SageMaker CreateTrainingJob API](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html)는 SageMaker Python SDK의 [상위 레벨 'Estimator' 클래스](https://sagemaker.readthedocs.io/en/stable/overview.html)가 단순화하는 데 도움이 되는 여러 저수준 세부 사항을 필요로 합니다. 특히:
- 정확한 컨테이너 이미지 URI를 지정하는 대신, SDK가 선택된 프레임워크와 버전을 기반으로 이를 찾아줍니다
- SDK는 우리의 `scripts` 번들을 자동으로 압축하여 S3에 업로드하고, 훈련 작업이 거기서 로드하도록 구성합니다

따라서 먼저, 작업을 구성하고 어떤 인프라(컴퓨팅 인스턴스의 수와 유형)에서 실행되어야 하는지를 지정하는 `estimator` 객체를 생성합니다:

> ℹ️ 다른 작업을 대신 실행하는 서비스들처럼, 훈련 작업은 S3의 입력 훈련 데이터와 같은 리소스에 접근할 수 있도록 [IAM 역할](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)을 가정합니다. SageMaker 노트북 자체가 이미 가정된 역할로 실행되고 있으므로, 단순화를 위해 훈련 작업 역할을 노트북 역할과 동일하게 설정할 것입니다.


In [11]:
from sagemaker.huggingface.estimator import HuggingFace as HuggingFaceEstimator

nb_role = sagemaker.get_execution_role()

estimator = HuggingFaceEstimator(
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version="py39",

    source_dir="scripts",  # Local folder where fine-tuning script is stored
    entry_point="train.py",  # Actual script the training job should run

    base_job_name="news-classifier",  # Prefix for the training job name (timestamp will be added)
    instance_count=1,  # Number of instances train on (need to prepare your script for using >1!)
    # instance_type="ml.p3.2xlarge",  # Type of compute instance to use: p* and g* include GPUs
    instance_type="ml.g5.4xlarge",  # Type of compute instance to use: p* and g* include GPUs
    role=nb_role,  # IAM role the job will use to access AWS resources (e.g. data on S3)

    hyperparameters=hyperparameters,   # Training job parameters, as we set up earlier
    metric_definitions=metric_definitions,  # RegEx to extract metric data from training job logs
)

구성이 완료되면, 입력 데이터 위치를 지정하고 `estimator.fit()`을 실행하여 실제 훈련 작업을 시작할 수 있습니다.

데이터 입력 "채널"의 수, 이름 및 유형은 [사용자가 결정](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateTrainingJob.html#sagemaker-CreateTrainingJob-request-InputDataConfig)할 수 있습니다: 단지 노트북이 스크립트에서 기대하는 것과 동일한 채널을 구성하도록 하면 됩니다.

In [12]:
%%time

estimator.fit(
    {
        "train": train_s3_uri,
        "test": test_s3_uri,
    },
    wait=True,  # Wait for the training to complete (default=True)
    logs=True,  # Stream training job logs to the notebook (default=True, requires wait=True)
)

[12/17/24 03:16:30] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=712988;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=986509;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=287746;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=334945;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/image_uris.py#679\679]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name:                                       ]8;id=421460;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=347726;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             news-classifier-2024-12-17-03-16-30-011                                               

2024-12-17 03:16:30 Starting - Starting the training job
2024-12-17 03:16:30 Pending - Training job waiting for capacity......
2024-12-17 03:17:07 Pending - Preparing the instances for training...
2024-12-17 03:17:49 Downloading - Downloading the training image.....................
2024-12-17 03:21:11 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.9/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.9/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.T

> ⏰ 이 훈련 작업은 완료하는 데 약 10분이 소요되지만, '로컬' 모델보다 상당히 높은 정확도에 도달할 것입니다

CPU만 있는 작은 노트북 대신 GPU 가속 인스턴스에서 실행되기 때문에, 훈련 자체는 이전 '로컬' 예제보다 훨씬 더 빠를 것입니다. 하지만 작업이 인프라를 프로비저닝하고 시작하는 데 몇 분 정도 걸릴 수 있습니다.

현재 및 과거 작업의 상태는 [AWS Console for Amazon SageMaker](https://console.aws.amazon.com/sagemaker/home?#/jobs)의 *Training jobs* 페이지와 SageMaker Studio의 **Experiments** UI(왼쪽 사이드바의 🏠 **Home** 버튼에서)에서도 확인할 수 있습니다.

🟢 대기하고 로그를 스트리밍하는 기본 동작이 로컬과 비슷한 경험을 제공하지만, 훈련 작업은 노트북에 의존하지 않습니다:
- 노트북의 연결이 끊어지거나 종료되어도 훈련 작업은 계속됩니다
- `wait=False`를 설정하면 노트북에서 여러 훈련 작업을 병렬로 시작할 수 있습니다
- 재시작된 노트북을 이전 훈련 작업에 연결해야 할 경우, 아래와 같이 훈련 작업 이름으로 `.attach()`할 수 있습니다:

In [13]:
# estimator = HuggingFaceEstimator.attach("news-hf-2020-01-01-12-00-00-000")

훈련 작업이 완료되면, 컨테이너의 모델 출력 폴더의 내용이 자동으로 S3에 아카이브됩니다.

아래와 같이 이 파일을 참조할 수 있으며, 비슷한 tar 압축 형식으로 준비하여 SageMaker 외부에서 훈련된 모델을 배포용으로 가져올 수도 있습니다:

In [14]:
estimator.latest_training_job.describe()["ModelArtifacts"]["S3ModelArtifacts"]

's3://sagemaker-us-east-1-057716757052/news-classifier-2024-12-17-03-16-30-011/output/model.tar.gz'

## 모델을 추론에 사용하기

모델 훈련이 완료되면, 새로운 데이터에 대한 추론을 수행할 준비가 된 것입니다.
SageMaker는 [온디맨드 추론을 위한 모델 배포](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html) 또는 [배치 추론 작업 실행](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html)을 위한 여러 완전 관리형 옵션을 제공합니다.

> ℹ️ **기억하세요:** 사용 사례에 맞는 적절한 추론 옵션을 선택하세요 - 단순히 배치 데이터를 처리하고 싶다면 실시간 엔드포인트를 배포할 필요가 없습니다!
>
> 지금까지 사용해온 동일한 상위 레벨 SageMaker Python SDK를 통해 배치 추론을 실행하는 방법에 대한 자세한 내용은 [SageMaker Batch Transform 사용하기](https://sagemaker.readthedocs.io/en/stable/overview.html#sagemaker-batch-transform)를 참조하세요.

이 예제에서는 모델을 [실시간 추론 엔드포인트](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html)에 배포할 것입니다. 이를 통해 온디맨드로 헤드라인을 분류할 수 있습니다.

다시 한 번 엔드포인트를 실행할 인프라 유형을 지정할 것입니다. 따라서 시작하는 데 몇 분이 걸릴 것입니다. 이 테스트 엔드포인트는 매우 적은 트래픽을 처리할 것이므로, 훈련 때와는 **다른 유형**의 인스턴스를 사용할 수 있습니다 - 더 작고 저렴한 인프라를 사용할 수 있습니다:

In [15]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
)

[12/17/24 03:28:23] INFO     Creating model with name: news-classifier-2024-12-17-03-28-23-804      ]8;id=701747;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=471083;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[12/17/24 03:28:24] INFO     Creating endpoint-config with name                                     ]8;id=920861;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=481125;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             news-classifier-2024-12-17-03-28-23-804                                               

                    INFO     Creating endpoint with name news-classifier-2024-12-17-03-28-23-804    ]8;id=276675;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=52367;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

-------!

배포 후에는 [AWS Console for Amazon SageMaker](https://console.aws.amazon.com/sagemaker/home?#/endpoints)의 *Endpoints* 페이지와 SageMaker Studio UI의 **Deployments > Endpoints** 섹션(왼쪽 사이드바의 🏠 **Home** 버튼에서)에서 엔드포인트를 찾을 수 있습니다.

훈련 작업과 마찬가지로, 엔드포인트는 노트북 자체와 분리되어 있습니다. 다음과 같이 이전에 배포된 엔드포인트에 노트북을 연결할 수 있습니다:

In [16]:
# from sagemaker.huggingface import HuggingFacePredictor
# predictor = HuggingFacePredictor("news-classifier-2023-03-24-13-31-09-895")

## 이제 모델이 RESTful API로 프로덕션 환경에 배포되었습니다!

[Predictor](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html)는 여기서 모델을 메모리에 로드하지 않고, 대신 배포된 엔드포인트에 대한 HTTPS API 호출을 래핑합니다.

여기서는 Hugging Face 프레임워크에서 제공하는 기본 `application/json` 직렬화를 사용하고 있지만, 다른 프레임워크들은 서로 다른 기본 형식을 가지고 있으며, 사용자 지정 [serializers](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html)와 [deserializers](https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html)(클라이언트/`predictor` 측)와 사용자 지정 [`input_fn`과 `output_fn`](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#process-model-input)(엔드포인트 컨테이너 측)을 사용하여 원하는 거의 모든 요청 또는 응답 형식을 설정할 수 있습니다: 서빙 스택을 처음부터 직접 작성할 필요가 없습니다.

요청 직렬화/역직렬화 및 처리는 [HuggingFacePredictor](https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html#hugging-face-predictor)와 미리 빌드된 추론 컨테이너에 의해 처리되므로, 노트북에서 배포된 모델을 호출하는 것은 로컬 인메모리 모델을 호출하는 것만큼 쉽습니다:

In [17]:
def classify(text: str) -> dict:
    """Classify a headline and print the results"""
    return predictor.predict({"inputs":[text]})[0]


# Either try out the interactive widget:
interaction = widgets.interact_manual(
    classify,
    text=widgets.Text(
        value="The markets were bullish after news of the merger",
        placeholder="Type a news headline...",
        description="Headline:",
        layout=widgets.Layout(width="99%"),
    ),
)
interaction.widget.children[1].description = "Classify!"

interactive(children=(Text(value='The markets were bullish after news of the merger', continuous_update=False,…

Alternatively (if e.g. you're struggling with the UI widget library), you can call the endpoint direct from code:

In [18]:
classify("Retailers are expanding after the recent economic growth")

{'label': 'Business', 'score': 0.9933713674545288}

## 정리

SageMaker 작업(훈련, 처리, 배치 추론 등)은 실행되는 동안만 온디맨드 컴퓨팅을 사용하지만, 배포된 실시간 추론 엔드포인트는 종료할 때까지 계속해서 리소스를 소비합니다.

실험이 끝나면, 불필요한 비용을 피하기 위해 더 이상 필요하지 않은 엔드포인트를 삭제하세요:

In [19]:
# predictor.delete_endpoint(delete_endpoint_config=True)

## 검토

이 노트북에서는 Amazon SageMaker에서 Hugging Face transformers를 사용하여 텍스트 분류 모델을 훈련하고 배포하는 방법을 보여주었습니다.

[Headline Classifier Local 노트북](Headline%20Classifier%20Local.ipynb)과 비교했을 때 이 접근 방식의 장점은 다음과 같습니다:
- 훈련 작업 기간 **동안만** 전문 컴퓨팅 리소스(예: 고성능 또는 GPU 가속 인스턴스)를 자동으로 프로비저닝할 수 있습니다: 리소스가 활용도가 낮은 상태로 방치되지 않으면서 훈련에서 좋은 성능을 얻을 수 있습니다
- 사용자가 무엇이 효과가 있었고 무엇이 효과가 없었는지 기록해야 하는 로컬 노트북 실험과 달리, 훈련 작업의 이력(매개변수, 메트릭, 출력 등 포함)이 자동으로 추적됩니다
- 훈련된 모델을 SDK 호출 한 번으로 안전한 프로덕션 준비된 웹 엔드포인트에 배포할 수 있습니다: 동작을 깊이 사용자 지정하고 싶지 않다면 컨테이너나 웹 애플리케이션 패키징이 필요 없습니다

로컬 노트북과 이 SageMaker 버전, 그리고 동봉된 [scripts/train.py](scripts/train.py) 스크립트 파일을 비교함으로써, 자신의 또는 오픈 소스 노트북 내 ML 워크플로우를 SageMaker "스크립트 모드" 훈련 작업으로 마이그레이션하는 방법에 대한 아이디어를 얻을 수 있습니다.

이 워크샵의 다음 "마이그레이션 챌린지" 연습에서는 다른 "로컬" 노트북에 대해 이 프로세스를 직접 반복해볼 것입니다.

또한 SageMaker 작업을 파이프라인으로 연결하고 CI/CD로 워크플로우를 자동화하는 등의 추가 단계를 보여주는 [aws-samples/amazon-sagemaker-from-idea-to-production](https://github.com/aws-samples/amazon-sagemaker-from-idea-to-production)에도 관심이 있을 수 있습니다.